In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
import joblib
from tqdm import tqdm
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from preprocessing.noise_reduction import NoiseReducer
from preprocessing.silence_removal import SilenceRemover
from preprocessing.speech_filter import SpeechFilter
from preprocessing.identity import Identity


from audio import Audio

original_metadata_path = os.path.join(".", "data", "original_data_labeled.tsv")
filtered_metadata_path = os.path.join(".", "data", "filtered_data_labeled.tsv")
audio_dir = os.path.join(".", "data", "filtered_clips")

### Pipeline

In [2]:

folder_path = 'trials/features/all_features_2000_n'
X_loaded = joblib.load(os.path.join(folder_path, 'X.joblib'))
y_loaded = joblib.load(os.path.join(folder_path, 'y.joblib'))

print(X_loaded.shape)
print(y_loaded.shape)

(8000, 313)
(8000,)


In [3]:
def load_and_concatenate_features(features, main_folder='extracted_features'):
    """
    Load and concatenate features from a single folder.
    
    Args:
        folders (list): List of feature names (used for validation)
        main_folder (str): Main directory containing all features
        
    Returns:
        tuple: (X_train_combined, X_test_combined, y_train, y_test)
    """
    X_train_arrays = []
    X_test_arrays = []
    
    # Load y labels only once since they're the same for all features
    y_train = joblib.load(os.path.join(main_folder, 'y_train.joblib'))
    y_test = joblib.load(os.path.join(main_folder, 'y_test.joblib'))
    
    for feature in features:
        # Construct file names for train and test data
        train_file = f"{feature}_train.joblib"
        test_file = f"{feature}_test.joblib"
        
        # Validate existence of files
        train_path = os.path.join(main_folder, train_file)
        test_path = os.path.join(main_folder, test_file)
        
        if not os.path.exists(train_path) or not os.path.exists(test_path):
            raise ValueError(f"Missing train or test file for feature: {feature}")
        
        # Load train and test data
        X_train = joblib.load(train_path)
        X_test = joblib.load(test_path)
        
        if X_train.shape == np.ravel(X_train).shape:
            X_train = X_train.reshape(X_train.shape[0], -1)
            X_test = X_test.reshape(X_test.shape[0], -1)
            print(f"Reshaped X_train for {feature}: {X_train.shape}")
        X_train_arrays.append(X_train)
        X_test_arrays.append(X_test)
    
    # Concatenate along feature axis (axis=1)
    X_train_combined = np.concatenate(X_train_arrays, axis=1)
    X_test_combined = np.concatenate(X_test_arrays, axis=1)
    
    return X_train_combined, X_test_combined, y_train, y_test

In [4]:
x_train_combined, x_test_combined, y_train, y_test = load_and_concatenate_features(['mfcc150','hfcc150','cpps','fO_1','jitter','pitch_range_1','pitch_mean_min_max_freq_3','alpha_ratio1','spectral5'],
main_folder='trials/features/all_classes_7_ft_splitted')


print(f"x_train_combined shape: {x_train_combined.shape}")
print(f"x_test_combined shape: {x_test_combined.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

Reshaped X_train for alpha_ratio1: (145504, 1)
x_train_combined shape: (145504, 313)
x_test_combined shape: (36376, 313)
y_train shape: (145504,)
y_test shape: (36376,)


In [5]:
X_train_loaded = x_train_combined
X_test_loaded = x_test_combined
y_train_loaded = y_train % 2
y_test_loaded = y_test % 2
print(X_train_loaded.shape, X_test_loaded.shape, y_train_loaded.shape, y_test_loaded.shape)

(145504, 313) (36376, 313) (145504,) (36376,)


In [3]:

def display_highest_accuracy(highest_accuracy, best_combination):
    """
    Display the highest accuracy and its corresponding feature combination in a styled Markdown block.

    Args:
        highest_accuracy (float): The highest accuracy achieved.
        best_combination (list): The best feature combination.
    """
    display(Markdown(f"""
<div style="border: 2px solid #424242; padding: 15px; border-radius: 12px; background-color: #212121; color: #E0E0E0;">
    <h2 style="color: #FFCA28; text-align: center; font-family: Arial, sans-serif;">🌟 Current Highest Accuracy 🌟</h2>
    <p style="font-size: 18px; text-align: center; font-family: Arial, sans-serif;">
        <strong>Accuracy:</strong> <span style="color: #76FF03; font-weight: bold;">{highest_accuracy:.6f}</span><br>
        <strong>Combination:</strong> <span style="color: #29B6F6; font-weight: bold;">{', '.join(best_combination)}</span>
    </p>
</div>
"""))



In [ ]:
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from IPython.display import display, Markdown


def find_best_feature_combination(X, y, feature_indices):
    """
    Find the best feature combination for classification, including non-consecutive combinations, and store all accuracies.

    Args:
        X (numpy.ndarray): Feature vectors.
        y (numpy.ndarray): Labels.
        feature_indices (list of tuples): List of (start, end) indices for each feature vector.

    Returns:
        dict: Best combination, accuracy, feature indices, and all accuracies.
    """
    best_accuracy = 0
    best_combination = None
    best_features = None
    all_accuracies = []

    # Generate all possible feature combinations
    print("Generating all possible feature combinations...")
    print(f"Feature indices: {feature_indices}")
    total_combinations = sum(len(list(combinations(feature_indices, r))) for r in range(1, len(feature_indices) + 1))
    print(f"Total combinations to evaluate: {total_combinations}")
    with tqdm(total=total_combinations, desc="Evaluating feature combinations") as pbar:
        for r in range(1, len(feature_indices) + 1):
            for combination in combinations(feature_indices, r):
                # Extract features for the current combination
                selected_features = []
                for start, end in combination:
                    selected_features.append(X[:, start:end])
                X_combined = np.hstack(selected_features)

                # Perform cross-validation
                clf = SVC(kernel='rbf', C=1000, gamma=0.0001, random_state=42, class_weight='balanced')
                cv_scores = cross_val_score(clf, X_combined, y, cv=5, scoring='accuracy')

                # Calculate the mean cross-validation score
                mean_cv_score = np.mean(cv_scores)

                # Store the mean cross-validation score and combination
                all_accuracies.append({"combination": combination, "accuracy": mean_cv_score})

                # Update the best combination if current accuracy is higher
                if mean_cv_score > best_accuracy:
                    best_accuracy = mean_cv_score
                    best_combination = combination
                    best_features = X_combined

                
                pbar.update(1)

    return {
        "best_combination": best_combination,
        "best_accuracy": best_accuracy,
        "best_features": best_features,
        "all_accuracies": all_accuracies
    }

feature_indices = [(0, 75), (75,150),(150, 300), (300, 301), (301, 302), (302, 307), (307, 308), (308, 309), (309,310), (310,312)]  # Example feature vector indices
mapping = {
    "mfcc75mean": (0, 75),
    "mfcc75std": (75, 150),
    "hfcc150": (150, 300),
    "fundamental_freq": (300, 301),
    "jitter": (301, 302),
    "spectral": (302, 307),
    "alpha_ratio": (307, 308),
    "cpps": (308, 309),
    "pitch_range": (309, 310),
    "mean_min_max_freq": (310, 312)
}
result = find_best_feature_combination(X_loaded, y_loaded, feature_indices)
print("Best Combination:", result["best_combination"])
print("Best Accuracy:", result["best_accuracy"])
print("All Accuracies:", result["all_accuracies"])
#display_highest_accuracy(result["best_accuracy"], result["best_combination"])



Generating all possible feature combinations...
Feature indices: [(0, 75), (75, 150), (150, 300), (300, 301), (301, 302), (302, 307), (307, 308), (308, 309), (309, 310), (310, 312)]
Total combinations to evaluate: 1023


Evaluating feature combinations:  55%|█████▍    | 558/1023 [59:17<28:47,  3.71s/it]  d:\ProgramFiles\Python312\Lib\site-packages\sklearn\mixture\_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
Evaluating feature combinations:  78%|███████▊  | 800/1023 [1:41:10<43:12, 11.63s/it]  d:\ProgramFiles\Python312\Lib\site-packages\sklearn\mixture\_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(
Evaluating feature combinations:  84%|████████▎ | 856/1023 [1:50:21<21:31,  7.74s/it]  


KeyboardInterrupt: 

In [ ]:
# Sort all accuracies in descending order
sorted_accuracies = sorted(result["all_accuracies"], key=lambda x: x["accuracy"], reverse=True)

# Write the sorted accuracies to a text file in tabular form with improved formatting
with open("statistics/sorted_accuracies_2000_GMM.txt", "w") as f:
    # Write the header with borders
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")
    f.write(f"| {'Combination':<98} | {'Accuracy':<10} |\n")
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")
    
    # Write each combination and its accuracy with borders
    for entry in sorted_accuracies:
        combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
        f.write(f"| {combination_str:<98} | {entry['accuracy']:<10.6f} |\n")
    f.write("|" + "=" * 100 + "|" + "=" * 12 + "|\n")

# Display the sorted combinations with their accuracies
for entry in sorted_accuracies:
    combination_str = ", ".join([name for name, indices in mapping.items() if indices in entry['combination']])
    print(f"Combination: {combination_str}, Accuracy: {entry['accuracy']}")

Combination: mfcc75mean, pitch_range, Accuracy: 0.58625
Combination: mfcc75mean, jitter, pitch_range, Accuracy: 0.58525
Combination: mfcc75mean, Accuracy: 0.5831250000000001
Combination: mfcc75mean, spectral, Accuracy: 0.583
Combination: mfcc75mean, spectral, cpps, pitch_range, Accuracy: 0.5827500000000001
Combination: mfcc75mean, spectral, pitch_range, Accuracy: 0.58275
Combination: mfcc75mean, fundamental_freq, Accuracy: 0.582375
Combination: mfcc75mean, jitter, spectral, pitch_range, Accuracy: 0.58175
Combination: mfcc75mean, fundamental_freq, pitch_range, Accuracy: 0.5817499999999999
Combination: mfcc75mean, cpps, pitch_range, Accuracy: 0.5815
Combination: mfcc75mean, jitter, alpha_ratio, pitch_range, Accuracy: 0.5814999999999999
Combination: mfcc75mean, alpha_ratio, pitch_range, Accuracy: 0.581375
Combination: mfcc75mean, jitter, Accuracy: 0.58125
Combination: mfcc75mean, jitter, cpps, pitch_range, Accuracy: 0.58125
Combination: mfcc75mean, fundamental_freq, jitter, Accuracy: 0.58

In [ ]:
from playsound import playsound

def play_sound(file_path):
    """
    Play a sound file.

    Args:
        file_path (str): Path to the sound file.
    """
    try:
        playsound(file_path)
        playsound(file_path)
    except Exception as e:
        print(f"Error playing sound: {e}")

play_sound("../sound/done.mp3")